We already have testbenched the crop extensively - assume it works. 

What we want now is data that, when max-normalized in ap_fixed<16,11>, won't just go to zero.

In [1]:
import os
import copy
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import h5py
from sklearn.preprocessing import StandardScaler
from scipy.optimize import least_squares

np.random.seed(0)


Globals

In [2]:
FP_TOTAL = 16
FP_INT = 10
FP_FRAC = FP_TOTAL-FP_INT-1
LUT_BITS = FP_TOTAL-1 # number of bits to index LUT
NUM_ROWS = 2**LUT_BITS

Helper functions

In [3]:
def float_to_fixed_point(value: float, total_bits: int, fractional_bits: int) -> str:
    """
    Convert a float to a fixed-point binary representation.
    
    :param value: Float value to convert.
    :param total_bits: Total number of bits for the representation.
    :param fractional_bits: Number of fractional bits.
    :return: Fixed-point binary string.
    """
    scale = 2 ** fractional_bits
    fixed_point_value = int(round(value * scale))
    
    # Handle two's complement representation for negative numbers
    if fixed_point_value < 0:
        fixed_point_value = (1 << total_bits) + fixed_point_value
    
    return f"{fixed_point_value:0{total_bits}b}"


def fixed_point_to_float(binary: str, fractional_bits: int) -> float:
    """
    Convert a fixed-point binary representation to a float.
    
    :param binary: Fixed-point binary string.
    :param fractional_bits: Number of fractional bits.
    :return: Float value.
    """
    total_bits = len(binary)
    int_value = int(binary, 2)
    
    # Handle two's complement for negative numbers
    if int_value >= (1 << (total_bits - 1)):
        int_value -= (1 << total_bits)
    
    return int_value / (2 ** fractional_bits)

In [4]:
# for val_int in range(1,10): #range(2**FP_TOTAL):
reciprocal_LUT = {}
for val_int in range(1,2**(LUT_BITS)):

    val_bin = float_to_fixed_point(val_int, LUT_BITS, 0)

    val_float = fixed_point_to_float(val_bin + "0"*(FP_TOTAL-LUT_BITS), FP_FRAC)
    val_reciprocal_float = 1/val_float
    val_reciprocal_fixed = float_to_fixed_point(val_reciprocal_float, FP_TOTAL, FP_FRAC)

    # print(f"val_bin: {val_bin}, val_float: {val_float}, val_reciprocal_float: {val_reciprocal_float}, val_reciprocal_fixed: {val_reciprocal_fixed}")
    reciprocal_LUT[val_bin] = val_reciprocal_fixed


    val_bin = float_to_fixed_point(-val_int, LUT_BITS, 0)

    val_float = fixed_point_to_float(val_bin + "0"*(FP_TOTAL-LUT_BITS), FP_FRAC)
    val_reciprocal_float = 1/val_float
    val_reciprocal_fixed = float_to_fixed_point(val_reciprocal_float, FP_TOTAL, FP_FRAC)

    reciprocal_LUT[val_bin] = val_reciprocal_fixed

In [5]:
rtl_dir = "04_ref_design"
template_fpath = os.path.join(rtl_dir, "division_LUT_template.sv")
with open(template_fpath, "r") as f:
    template = f.read()

In [6]:
module_rtl = template.replace("{FP_TOTAL}", str(FP_TOTAL))
module_rtl = module_rtl.replace("{FP_TOTAL-1}", str(FP_TOTAL-1))
module_rtl = module_rtl.replace("{FP_INT}", str(FP_INT))
module_rtl = module_rtl.replace("{NUM_ROWS-1}", str(NUM_ROWS-1))
module_rtl = module_rtl.replace("{LUT_BITS}", str(LUT_BITS))
module_rtl = module_rtl.replace("{LUT_BITS-1}", str(LUT_BITS-1))
module_rtl_top, module_rtl_bottom = module_rtl.split("// INSERT LUT HERE")
module_rtl_lines = [module_rtl_top]

for k,v in reciprocal_LUT.items():
    if v[0]=="1": continue
    line = f"        {LUT_BITS}'b{k} : reciprocal = {FP_TOTAL}'b{v};\n"
    if v=="0"*FP_TOTAL:
        line = f"        default : reciprocal = {FP_TOTAL}'b{v};\n"
        module_rtl_lines.append(line)
        break
        
    module_rtl_lines.append(line)

for k,v in reciprocal_LUT.items():
    if v[0]=="0": continue
    line = f"        {LUT_BITS}'b{k} : reciprocal = {v};\n"
    module_rtl_lines.append(line)

module_rtl_lines.append(module_rtl_bottom)

module_rtl = "".join(module_rtl_lines)

In [7]:
module_fpath = os.path.join(rtl_dir, f"division_LUT_ap_fixed_{FP_TOTAL}_{FP_INT}_s{LUT_BITS}.sv")
with open(module_fpath, "w") as f:
    f.write(module_rtl)

In [8]:
den_test_values = [5, 7, 11, 17, 31, 41, 43, 47, 61,  67]
den_test_bin_values = [float_to_fixed_point(i, FP_TOTAL, FP_FRAC) for i in den_test_values]

print(f"den_test_bin_values:")
for i in den_test_bin_values: print(i)


den_test_bin_values:
0000000010100000
0000000011100000
0000000101100000
0000001000100000
0000001111100000
0000010100100000
0000010101100000
0000010111100000
0000011110100000
0000100001100000


In [9]:
den_test_int_values = [int(fixed_point_to_float(float_to_fixed_point(i, FP_TOTAL, FP_FRAC),0)) for i in den_test_values]
out_test_int_values = [int(fixed_point_to_float(float_to_fixed_point(1/i, FP_TOTAL, FP_FRAC),0)) for i in den_test_values]

print(f"den_test_int_values: {den_test_int_values}")
print(f"out_test_int_values: {out_test_int_values}")

den_test_int_values: [160, 224, 352, 544, 992, 1312, 1376, 1504, 1952, 2144]
out_test_int_values: [6, 5, 3, 2, 1, 1, 1, 1, 1, 0]


In [10]:
a_test = 67
b_test = 12
c_test = a_test*b_test

print(f"den_test: {float_to_fixed_point(a_test, FP_TOTAL, FP_FRAC)} := int {int(fixed_point_to_float(float_to_fixed_point(a_test, FP_TOTAL, FP_FRAC), 0))}")
print(f"out_test: {float_to_fixed_point(b_test, FP_TOTAL, FP_FRAC)} := int {int(fixed_point_to_float(float_to_fixed_point(b_test, FP_TOTAL, FP_FRAC), 0))}")
print(f"out_test: {float_to_fixed_point(c_test, FP_TOTAL, FP_FRAC)} := int {int(fixed_point_to_float(float_to_fixed_point(c_test, FP_TOTAL, FP_FRAC), 0))}")



den_test: 0000100001100000 := int 2144
out_test: 0000000110000000 := int 384
out_test: 0110010010000000 := int 25728


In [11]:
a_test_values = den_test_values[:6]
b_test_values = [den_test_values[-i]-3 for i in range(len(a_test_values))]
c_test_values = [i*j for (i,j) in zip(a_test_values, b_test_values)]

print(f"a_test_values: {a_test_values}")
print(f"b_test_values: {b_test_values}")
print(f"c_test_values: {c_test_values}")


a_test_values: [5, 7, 11, 17, 31, 41]
b_test_values: [2, 64, 58, 44, 40, 38]
c_test_values: [10, 448, 638, 748, 1240, 1558]


In [12]:
a_test_bin_values = [float_to_fixed_point(a, FP_TOTAL, FP_FRAC) for a in a_test_values]
b_test_bin_values = [float_to_fixed_point(b, FP_TOTAL, FP_FRAC) for b in b_test_values]
c_test_bin_values = [float_to_fixed_point(c, FP_TOTAL, FP_FRAC) for c in c_test_values]

print(f"a_test_bin_values:")
for a in a_test_bin_values: print(a) 

print("\n\n")
print(f"b_test_bin_values:")
for b in b_test_bin_values: print(b)


a_test_bin_values:
0000000010100000
0000000011100000
0000000101100000
0000001000100000
0000001111100000
0000010100100000



b_test_bin_values:
0000000001000000
0000100000000000
0000011101000000
0000010110000000
0000010100000000
0000010011000000


In [13]:
a_test_int_values = [int(fixed_point_to_float(a, 0)) for a in a_test_bin_values]
b_test_int_values = [int(fixed_point_to_float(b, 0)) for b in b_test_bin_values]
c_test_int_values = [int(fixed_point_to_float(c, 0)) for c in c_test_bin_values]

print(f"a_test_int_values:")
for a in a_test_int_values: print(a) 

print("\n\n")
print(f"b_test_int_values:")
for b in b_test_int_values: print(b)

print("\n\n")
print(f"c_test_int_values:")
for c in c_test_int_values: print(c)


a_test_int_values:
160
224
352
544
992
1312



b_test_int_values:
64
2048
1856
1408
1280
1216



c_test_int_values:
320
14336
20416
23936
-25856
-15680
